# NCME 2023 Data Science Upskilling Workshop - Session 3

# Data Wrangling

Questions contact: jhao@ets.org


# Parsing unstructured data

### step 1. load packages

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

### step 2. open the file

In [ ]:
with open('data_for_session_3/unstructured_example_log.txt') as f:
    txt = f.readlines()

In [ ]:
txt

In [ ]:
txt[0].strip()

### step 3. clean the lines by stripping off the \n

In [ ]:
txt = [t.strip() for t in txt]

In [ ]:
txt[0]

In [ ]:
txt[0].split(']')[0].replace('[','')

### step 4. chuncking and formatting

In [ ]:
txt[0].split(']')[0].replace('[','')

In [ ]:
txt[0].split(']')[1]

In [ ]:
txt[0].split(']')[1].upper()

### step 5. datetime conversion

In [ ]:
s = txt[0].split(']')[0].strip('[')

In [ ]:
s

In [ ]:
dtfmt ='%m/%d/%Y %I:%M:%S %p'   # %H -> 24 hours, %I-> 12 hours

In [ ]:
dt = datetime.strptime(s, dtfmt)

In [ ]:
dt

### step 6. convert into a dataframe

In [ ]:
col1 = []
col2 = []
col3 = []

In [ ]:
for line in txt:
    s1=line.split(']')[0].strip('[')
    dt = datetime.strptime(s1, dtfmt)
    col1.append(dt)
    s= line.split(']')[1].strip().split(':')
    col2.append(s[0])
    if len(s) == 2:
        col3.append(s[1])
    else:
        col3.append(np.nan)

In [ ]:
df = pd.DataFrame([col1,col2,col3]).T
df.columns=['datetime','event_name', 'event_result']

In [ ]:
df

In [ ]:
df.datetime - df.datetime.iloc[0]

### step 7. convert to seconds

In [ ]:
(df.datetime-df.datetime.loc[0]).apply(lambda x: x.total_seconds())

In [ ]:
df['time_seconds'] = (df.datetime-df.datetime.loc[0]).apply(lambda x: x.total_seconds())

In [ ]:
df

## Parsing JSON file

### Step 1. Loading the needed packages

In [ ]:
import pandas as pd
import json
from datetime import datetime

### Step 2. Open the unstructured data file and read it into a Python dictionary

In [ ]:
with open('data_for_session_3/structured_example_log.json') as f:
    txt = json.load(f)

In [ ]:
pd.DataFrame(txt.get('data'))

### Step 3. Convert a list of dictionaries to data frame

In [ ]:
pd.DataFrame(txt.get('data'))

## Parsing XML file

### Step 1. Load needed packages

In [ ]:
import xml.etree.ElementTree as et  # package for xml parsing

### Step 2. Specify XML file name

In [ ]:
xml_file_name = 'data_for_session_3/structured_example_log.xml'  # note that this xml is structured to comply with ETS' VPA data model

### Step 3. Parsing the xml tree

In [ ]:
tree = et.parse(xml_file_name)
root = tree.getroot()

In [ ]:
root

In [ ]:
# check how many child 
len(root)

In [ ]:
root[0]

In [ ]:
# find out the child of the root and the number of grandchild
for chd in root[0]:
    print(chd.tag,',',chd.text, ',',len(chd))

In [ ]:
# check the playerID child 
root[0][2]

In [ ]:
# check the child of playerID
for chd in root[0][2]:
    print(chd.tag,',',chd.text, ',',len(chd))

In [ ]:
# check the child of the child of playerID
root[0][2][0]

In [ ]:
# check the eventSequence child
root[0][5]

In [ ]:
for chd in root[0][5]:
    print(chd.tag,',',chd.text, ',',len(chd))

In [ ]:
# check the first event
root[0][5][0]

In [ ]:
for chd in root[0][5][0]:
    print(chd.tag,',',chd.text, ',',len(chd))

## The above codes show how you reach out to each leaf in the XML tree. You can put them together into a dataframe. 

## Visualization - Plotly express

In [ ]:
import plotly.express as px
#import pandas as pd

In [ ]:
# load data 
df = px.data.iris()

In [ ]:
df

In [ ]:
# scatter plot

px.scatter(df, x="sepal_width", y="sepal_length", color="species", size='species_id')

In [ ]:
# scatter plot 3D
px.scatter_3d(df, x="sepal_width", y="sepal_length", z="petal_length",color="species", size='species_id')

In [ ]:
# distribution
px.histogram(df,x='sepal_length',color='species')

In [ ]:
# distribution in facets
px.histogram(df,x='sepal_length',color='species',facet_col='species')

In [ ]:
# barplot

px.bar(df, y='sepal_length',color='species',facet_row='species')

In [ ]:
# scatter plot with trend line and marginalization

px.scatter(df, x="sepal_width", y="sepal_length", color="species", marginal_y="violin", \
           marginal_x="box", trendline="ols")

In [ ]:
# scatter matrix

px.scatter_matrix(df, dimensions=["sepal_width", "sepal_length", "petal_width", "petal_length"], color="species")

In [ ]:
# animation - using a different dataset

df = px.data.gapminder()

In [ ]:
df.head()

In [ ]:
px.scatter(df.query("year==2007"), x="gdpPercap", y="lifeExp", size="pop", color="continent",
           hover_name="country", log_x=True, size_max=60)

In [ ]:
px.scatter(df, x="gdpPercap", y="lifeExp", size="pop", color="continent",
           hover_name="country", log_x=True, size_max=60,animation_frame='year',range_y=[20,100])


## Ipywidgets for interaction

In [ ]:
from ipywidgets import interact, fixed, widgets
from IPython.display import display

In [ ]:
# let's use the GDP data, but you want to choose the contenient

def continent_gdp(df,conti):
    return px.scatter(df.query('continent==@conti'), x="gdpPercap", y="lifeExp", size="pop", color="country",
           hover_name="country", log_x=True, size_max=60,animation_frame='year',range_y=[20,100])


In [ ]:
continent_list = df.continent.unique().tolist()

In [ ]:
continent_list

In [ ]:
result = interact(continent_gdp, df=fixed(df), conti=continent_list)

### Some other widgets

In [ ]:
w = widgets.IntSlider()
display(w)

In [ ]:
w.value

In [ ]:
# now make it more complex
w = widgets.FloatSlider(
    value=7.5,
    min=0,
    max=10.0,
    step=0.1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='vertical',
    readout=True,
    readout_format='.1f',
)

In [ ]:
w

In [ ]:
w.value

In [ ]:
# check box widgets

w = widgets.Checkbox(
    value=False,
    description='Check me',
    disabled=False,
    indent=False
)

In [ ]:
w

In [ ]:
w.value

In [ ]:
# multiple selection
w = widgets.SelectMultiple(
    options=['Apples', 'Oranges', 'Pears','water mellon'],
    value=['Oranges'],
    #rows=10,
    description='Fruits',
    disabled=False
)

In [ ]:
w

In [ ]:
w.value

In [ ]:
# radio buttons

w = widgets.RadioButtons(
    options=['pepperoni', 'pineapple', 'anchovies'],
#    value='pineapple', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Pizza topping:',
    disabled=False
)

In [ ]:
w

In [ ]:
w.value

In [ ]:
# now using the radio button for the previous GDP plot

selector = widgets.RadioButtons(
    options=['Asia', 'Europe', 'Africa', 'Americas', 'Oceania'],
    description='Continent:',
    disabled=False
)

In [ ]:
result = interact(continent_gdp, df=fixed(df), conti=selector)